# Importing Libraries

In [ ]:
import re
import tweepy as tw
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

# Authenticate to Twitter

In [ ]:
consumer_key = ''
consumer_secret = ''
access_token = ''
access_token_secret = ''

In [ ]:
auth = tw.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tw.API(auth, wait_on_rate_limit=True)

# Get Tweets

In [ ]:
hashtag = "#Windows11"
query = tw.Cursor(api.search_tweets, q=hashtag).items(2000)
tweets = [{'Tweet':tweet.text} for tweet in query]

# Preprocessing Tweets

In [ ]:
df = pd.DataFrame.from_dict(tweets)
df

In [ ]:
# Import stopwords
import nltk
from nltk.corpus import stopwords

# Import textblob
from textblob import Word, TextBlob

In [ ]:
nltk.download('stopwords')
nltk.download('wordnet')
stop_words = stopwords.words('english')
custom_stopwords = ['RT', '#Windows11']

In [ ]:
def preprocess_tweets(tweet):
    processed_tweet = tweet
    processed_tweet.replace('[^\w\s]', '')
    processed_tweet = " ".join(word for word in processed_tweet.split() if word not in stop_words)
    processed_tweet = " ".join(word for word in processed_tweet.split() if word not in custom_stopwords)
    processed_tweet = " ".join(Word(word).lemmatize() for word in processed_tweet.split())
    processed_tweet = re.sub("[h]+[t]+[t]+[p]+[s]+[a-zA-Z0-9:_.\/\/]+","",processed_tweet)
    processed_tweet = re.sub("[@]+[a-zA-Z0-9:_.]+","",processed_tweet)
    return(processed_tweet)

df['Processed Tweet'] = df['Tweet'].apply(lambda x: preprocess_tweets(x))
df.head()

In [ ]:
df2 = df.drop_duplicates('Processed Tweet', keep='last')

# Calculate Polarity

In [ ]:
df2['polarity'] = df2['Processed Tweet'].apply(lambda x: TextBlob(x).sentiment[0])

In [ ]:
df2.head()

In [ ]:
def sentiment_tweets(pol):
    sent = ""
    if pol > 0.1:
        sent = "Positive"
    elif pol < -0.1:
        sent = "Negative"
    else:
        sent = "Neutral"
    return(sent)

df2['sentiment'] = df2['polarity'].apply(lambda x: sentiment_tweets(x))
df2.head()

In [ ]:
pos = df2['sentiment'].value_counts()['Positive']
neg = df2['sentiment'].value_counts()['Negative']
neut = df2['sentiment'].value_counts()['Neutral']
print('Positive \t:',pos,'\nNegative \t:',neg,'\nNeutral \t:',neut)

# Visualize

In [ ]:
data = [pos, neg, neut]
labels = ['Positive_tweets','Negative_tweets','Neutral_tweets']
plt.xticks(range(len(data)), labels)
plt.xlabel('Sentiment')
plt.ylabel('Number of Tweets')
plt.title('Sentiment Analysis')
plt.bar(range(len(data)), data)
plt.grid(color='#95a5a6', linestyle='--', linewidth=2, axis='y', alpha=0.7)
plt.show()
plt.show()

In [ ]:
df2.to_csv('/sentiment_analysis_output.csv')